## Hands-on session 2.5 - working with pre-built models

In this session we cover an example of a full workflow for ReMKiT1D. We will generate a partially built wrapper, extend it, and use it to produce physically relevant fluid simulations of a simplified Scrape-Off Layer. 

Demonstrated concepts: 

- Pre-built wrappers 
- Integration for finding equilibria
- Restarting runs
- Time-dependent terms 

In [2]:
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds 
import RMK_support.common_models as cm
import RMK_support.sk_normalization as skn

from simple_fluid import generatorSimpleFluid

import numpy as np
import holoviews as hv 
import panel as pn
import matplotlib.pyplot as plt

### The physical model 

We will look to model a 1D 2-fluid system with electrons and ions. We solve the following set of equations for both species

$$ \frac{\partial n_b}{\partial t} + \nabla (\Gamma_b) = S_b$$

$$ m_b\frac{\partial \Gamma_b}{\partial t} + \nabla (m_b \Gamma_b u_b) + q_b n_b E= R_{bb'} $$

$$ \frac{\partial W_b}{\partial t} + \nabla \left[(W_b + p_b)u_b + q_b \right] + j_b E= Q_b$$

where $S_b$ is the particle source (here due to implied ionization due to recycling), $R_{bb'}$ is the friction due to Coulomb collision between the two species, and $q_b$ is the Braginskii heat flux. $Q_b$ is the energy source due to any external sources and due to electron-ion thermal equilibration.

The electric field $E$ is obtained from what is effectively a current constraint $j_e+j_i = 0$ (by solving a degenerate Ampere-Maxwell equation). 

The boundary conditions are as follows:

- The upstream boundary (x=0) is reflective, meaning that all fluxes are set to 0 there
- The downstream sheath boundary has an outflow Bohm boundary condition on the particles $u_{b,sh} \ge c_s$, where $c_s$ is the ion acoustic speed.
- At the sheath, any outflow is converted into a recycling particle source $S_{rec} \propto \exp(-(L-x)/\lambda_{ion})$ where $\lambda_{ion}$ is an externally set ionization mean free path. The proportionality constant in front of the source is set so that it balances the flux of particles leaving the system - $\Gamma_{sh}$ (calculated from the Bohm condition above). A corresponding energy sink $Q = -\epsilon_{ion} S$ is added to the electron equation, mimicking the cooling of electrons due to ionization, with $\epsilon_{ion}$ being the user-set energy cost per ionization event.
- The energy outflow boundary condition is given as $q_{b,sh} = \gamma_b \Gamma_{sh} k T_{sh}$

The built-in energy source is given by some upstream energy flux $q_h$, spread uniformly over a number of cells upstream.

The initial conditions are specified by setting the upstream and downstream temperatures, as well as the upstream density. The profiles are then generated using the simple Two Point Model.

### Using pre-built wrapper generators 

In order to facilitate code sharing and flexibility, we encourage users to package their wrappers into parameterized generator functions using Python's kwargs feature. An example of a partial wrapper initialization is given in `simple_fluid.py`, and we will use this to generate the starting wrapper for this session

In [3]:
rk = generatorSimpleFluid(hdf5OutputFolder = "day_2_5",
                          mpiProcsX = 8,
                          Nx = 64, # Number of spatial cells - the grid is uniform
                          domainLength = 10.0, # Length of the domain in meters
                          Tu = 20.0, # Upstream initial temperature in eV
                          Td = 5.0, # Downstream initial temperature in eV
                          nu = 0.8, # Upstream initial density in 1e19m^-3
                          Nh = 1, # heating a single cell - this is the same as just setting an upstream inflow heat flux
                          heatingPower = 5.0, # Effective heating flux in MW/m^2
                          ionMFP = 0.5, # Ionization mfp in meters,
                          ionCost = 20.0 # Ionization cost in eV
                          );

Checking terms in model continuity-ne:
   Checking term divFlux
   Checking term rightBC
Checking terms in model continuity-ni:
   Checking term divFlux
   Checking term rightBC
Checking terms in model pressureGrad-Ge:
   Checking term bulkGrad
Checking terms in model pressureGrad-Gi:
   Checking term bulkGrad
Checking terms in model advection-Ge:
   Checking term divFlux
   Checking term rightBC
Checking terms in model advection-Gi:
   Checking term divFlux
   Checking term rightBC
Checking terms in model ampereMaxwell:
   Checking term currentGe_dual
   Checking term currentGi_dual
Checking terms in model lorentzForce:
   Checking term lorentzGe_dual
   Checking term lorentzGi_dual
Checking terms in model implicitTemp:
   Checking term identityTermTe
   Checking term wTermTe
   Checking term u2TermTe
   Checking term identityTermTi
   Checking term wTermTi
   Checking term u2TermTi
Checking terms in model advection-We:
   Checking term divFlux
Checking terms in model advection-Wi:
  

The generator above intentionally does not include time integration setups in order to allow extensions. 

### Time integration for finding equilibria

Let's assume we are only interested in the equilibrium solution of the above system, and do not care too much about the accuracy of time evolution. This is common in practice when looking for good equilibria for launching transients, which we shall examine later in this session. 

A way to reach the equilibrium relatively quickly is to employ an implicit time integration algorithm. In theory, a Backwards Euler step with infinite length is equivalent to solving the equilibrium problem. In practice, the solver has a maximum time step length after which it will (for one reason or another) break. To work around this, the `picardBDEIntegrator` has the `internalStepControl` option, which will instruct the integrator to reduce it's step length when it detects an integration failure, while periodically attempting to return to a single internal step. 

Let's choose a relatively long time step and set the internal step control option.

In [4]:
integrator = sc.picardBDEIntegrator(absTol=10.0, convergenceVars=["ne", "ni", "Ge_dual", "Gi_dual", "We", "Wi"], internalStepControl=True)

rk.addIntegrator("BE", integrator)

initialTimestep=101.0
rk.setIntegratorGlobalData(initialTimestep=initialTimestep)

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE", bdeStep.dict())

Since we're going for something close to equilibrium, let's ask the simulation to run for a relatively long time of 20000 normalized times (around 1.5ms in this case).

Set the output to be at every 1000 time units.

In [5]:
rk.setFixedNumTimesteps(1000)

We want to be able to restart the simulation, so let's ask the wrapper to keep a checkpoint every 50 steps using the `setRestartOptions` method on the wrapper. We only want to save checkpoints, not load any yet.

In [7]:
rk.setRestartOptions(save = True, frequency = 50)

### Create config 

To run the above, use the command 

```
mpirun -np 8 /home/stefan/SMijin/ReMKiT1D/build/src/executables/ReMKiT1D/ReMKiT1D | grep -E "Out|Int|Dump"
```

This will trim the output so we can see only the finished integration calls, when variable data is written to file, as well as when restart data is dumped. 

You will notice that the solver speeds up after a slow start.

In [8]:
rk.writeConfigFile()

### Data analysis 

In [9]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

numFiles=20
loadpath = rk.hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames,varsToIgnore=["ionGamma"])
loadedData

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

<xarray.Dataset>
Dimensions:     (x: 64, h: 1, v: 1, time: 21, dim_0: 1)
Coordinates:
  * x           (x) float64 0.07812 0.2344 0.3906 0.5469 ... 9.609 9.766 9.922
  * h           (h) int64 0
  * v           (v) float64 1.0
  * time        (time) float64 0.0 101.0 202.0 ... 1.818e+03 1.919e+03 2.02e+03
Dimensions without coordinates: dim_0
Data variables: (12/33)
    ne          (time, x) float64 0.8018 0.8054 0.8091 ... 2.778 2.896 2.385
    ne_dual     (time, x) float64 0.8036 0.8072 0.811 ... 2.837 2.64 2.129
    ni          (time, x) float64 0.8018 0.8054 0.8091 ... 2.778 2.896 2.385
    ni_dual     (time, x) float64 0.8036 0.8072 0.811 ... 2.837 2.64 2.129
    Ge_dual     (time, x) float64 0.0 0.0 0.0 0.0 ... 0.02009 0.02571 0.0
    Ge          (time, x) float64 0.0 0.0 0.0 0.0 ... 0.01818 0.0229 0.02851
    ...          ...
    gammaRight  (time, dim_0) float64 4.839 4.923 4.949 ... 4.895 4.854 4.824
    cs_b        (time, dim_0) float64 0.01221 0.01184 ... 0.01595 0.01602
    n_b         (time, dim_0) float64 2.863 1.377 1.185 ... 1.981 2.069 2.129
    G_b         (time, dim_0) float64 0.03496 0.0163 0.01479 ... 0.03301 0.0341
    u_b         (time, dim_0) float64 0.01221 0.01184 ... 0.01595 0.01602
    ionGamma    (time, dim_0) float64 2.5 2.5 2.5 2.5 2.5 ... 2.5 2.5 2.5 2.5

We can have a quick look to see that the simulation is converging

In [10]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)
dashboard.fluid2Comparison().show()

Launching server at http://localhost:34677


/usr/local/lib/python3.8/dist-packages/holoviews/core/spaces.py:497: ParamFutureWarning: Parameter 'callable' on <class 'holoviews.core.spaces.Callable'> is being given a valid parameter reference <function _param_bind.<locals>.wrapped at 0x7f330f5fc280> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function _param_bind.<locals>.wrapped at 0x7f330f5fc280> will be resolved to its underlying value :Curve   [time]   (ne). Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(callable=callable,
/usr/local/lib/python3.8/dist-packages/holoviews/core/spaces.py:497: ParamFutureWarning: Parameter 'callable' on <class 'holoviews.core.spaces.Callable'> is being given a valid parameter reference <function _param_bind.<locals>.wrapped at 0x7f330f5ce040> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <functio

### Adding a time-dependent perturbation 

Let's assume we're happy with how close the above simulation is to equilibrium, and that we now wish to perturb the heating power. 

One way of doing this would be changing the input power into the system and restarting the simulation. However, we might want a periodic perturbation. We will see how to add a periodic perturbation to our heating source, but first we must rebuild the wrapper.

In [11]:
rk = generatorSimpleFluid(hdf5OutputFolder = "day_2_5",
                          mpiProcsX = 8,
                          Nx = 64, # Number of spatial cells - the grid is uniform
                          domainLength = 10.0, # Length of the domain in meters
                          Tu = 20.0, # Upstream initial temperature in eV
                          Td = 5.0, # Downstream initial temperature in eV
                          nu = 0.8, # Upstream initial density in 1e19m^-3
                          Nh = 1, # heating a single cell - this is the same as just setting an upstream inflow heat flux
                          heatingPower = 5.0, # Effective heating flux in MW/m^2
                          ionMFP = 0.5, # Ionization mfp in meters,
                          ionCost = 20.0 # Ionization cost in eV
                          );

Checking terms in model continuity-ne:
   Checking term divFlux
   Checking term rightBC
Checking terms in model continuity-ni:
   Checking term divFlux
   Checking term rightBC
Checking terms in model pressureGrad-Ge:
   Checking term bulkGrad
Checking terms in model pressureGrad-Gi:
   Checking term bulkGrad
Checking terms in model advection-Ge:
   Checking term divFlux
   Checking term rightBC
Checking terms in model advection-Gi:
   Checking term divFlux
   Checking term rightBC
Checking terms in model ampereMaxwell:
   Checking term currentGe_dual
   Checking term currentGi_dual
Checking terms in model lorentzForce:
   Checking term lorentzGe_dual
   Checking term lorentzGi_dual
Checking terms in model implicitTemp:
   Checking term identityTermTe
   Checking term wTermTe
   Checking term u2TermTe
   Checking term identityTermTi
   Checking term wTermTi
   Checking term u2TermTi
Checking terms in model advection-We:
   Checking term divFlux
Checking terms in model advection-Wi:
  

### Time signal data

While it is possible to add explicit time dependence by composing different derivation objects, ReMKiT1D offers a built-in way of adding periodic time dependence to matrix terms by using the `TimeSignalData` class. 

The documentation covers the two currently available signal forms (a hat function and a cut sine). See the Usage section below

In [12]:
sc.TimeSignalData?

Init signature:
sc.TimeSignalData(
    signalType='none',
    period=0.0,
    params: List[float] = [],
    realPeriod=False,
) -> None
Docstring:      Container for custom term time dependence options
Init docstring:
TimeSignalData constructor

Args:
    signalType (str, optional): Type of time signal. Defaults to "none".
    period (float, optional): Period of time signal. Defaults to 0.0 and should be specified if type not "none".
    params (List[float], optional): Signal parameters. Defaults to [] and depend on time signal type.
    realPeriod (bool, optional): If true ReMKiT1D will interpret the period units as seconds. Defaults to False.

Usage:
    Available non-trivial signal types are "hat" and "cutSine", the first representing a box-shaped signal and the second half a sine period with an amplitude of 1. params(1) and params(2) should then represent the points in the period when the signal starts and ends, respectively, as a fraction of the period (e.g. [0.1,0.3] means that t

Let us create a `TimeSignalData` object with a "cutSine" profile, and a $100\mu \text{s}$ period, with the signal active over 30% of the period, and with a starting delay of $20\mu \text{s}$.

In [13]:
signalData = sc.TimeSignalData(signalType="cutSine", period = 100*1e-6, realPeriod = True, params = [0.2,0.5])

We will use the `simpleSourceTerm` from `common_models` to add a heating perturbation with the above time dependence

In [14]:
cm.simpleSourceTerm?

Signature:
cm.simpleSourceTerm(
    evolvedVar: str,
    sourceProfile: numpy.ndarray,
    timeSignal=<RMK_support.simple_containers.TimeSignalData object at 0x7f331beed7c0>,
) -> RMK_support.simple_containers.GeneralMatrixTerm
Docstring:
Simple implicit source term with given source profile

Args:
    evolvedVar (str): Name of evolved variable
    sourceProfile (np.ndarray): Spatial source profile
    timeSignal (sc.TimeSignalData): Optional time signal component of source. Defaults to constant signal.

Returns:
    sc.GeneralMatrixTerm: Term ready for adding to a model
File:      /usr/local/lib/python3.8/dist-packages/RMK_support/common_models.py
Type:      function

Let's add a fast perturbation to the electron heating. The electron energy density variable is "We", and we would like to add a term of the form

$$ \frac{\partial W_e}{\partial t} = Q(x,t) $$

where we would like to have $ Q(x,t) =   T(t)  X(t) q_{eff}/L_h$, with $q_{eff}$ being the effective perturbation flux amplitude, $L_h$ the heating length, and $X$ and $T$ dimensionless spatial and temporal profiles.

The `spatialProfile` argument to `simpleSourceTerm` should be set to the normalized value of $X(x)q_{eff}/L_h$. 

This example uses the default normalization, where energy density is normalized to $n_0 eT_0$, and time is normalized to $t_0$. 

We can use the `sk_normalization` module (specifically the `calculateNorms` function) to get the time normalization  by first querying the wrapper about its normalization values.

In [27]:
rk.normalization
densNorm = rk.normalization["density"]
tempNorm = rk.normalization["eVTemperature"]

Use the values from the above dictionary to get the time normalization using `calclateNorms`

In [28]:
normIn = rk.normalization
timeNorm = skn.calculateNorms(normIn["eVTemperature"], normIn["density"], normIn["referenceIonZ"])["time"]


Assuming that we want $q_{eff}$ to be in units of MW/$\text{m}^2$ and that $L_h$ is in meters, the equation with normalization taken out will look like 

$$ \frac{n_0 eT_0}{t_0}\frac{\partial W_e}{\partial t} = X(x) T(t) \frac{q_{eff}}{L_h}\times 10^6$$

Moving all of the normalization values to one side, we get that the `spatialProfile` after normalization should be 

$$ \frac{t_0 \times 10^6}{n_0 eT_0} X(x) \frac{q_{eff}}{L_h} $$ 

Assuming we want to heat only the first cell (as in the equilibrium case), we can calculate $L_h$ as

In [24]:
L = sum(rk.grid.xWidths) # this should be equal to the domain length we set
Nx = len(rk.grid.xWidths)
Lh = L/Nx

We can then create a numpy array for the spatial profile of the source

In [29]:
elCharge = 1.60218e-19 # electron charge for convenience

q_eff = 10.0 # Effective heating perturbation flux in MW/m^2


spatialProf = np.zeros(Nx)
spatialProf[0] = timeNorm * q_eff * 1e6 /(Lh*densNorm*elCharge*tempNorm) # [YOUR CODE HERE] Hint: only the first cell is being heated

Finally, we can add a model with the required source term. Use `simpleSourceTerm` with the spatial and temporal profiles you just generated to create a perturbation term, add it to a new model. Do not forget to add that model to the wrapper afterwards!

In [31]:
newModel = sc.CustomModel("perturbation")

newModel.addTerm("source",cm.simpleSourceTerm(evolvedVar="We",
                                              sourceProfile=spatialProf,
                                              timeSignal=signalData
                                              )
                 )

rk.addModel(newModel)

Checking terms in model perturbation:
   Checking term source


### Time integration resolving the perturbation

We now want to resolve a few periods of the perturbation, while also resolving electron conductive time scales.

In [32]:
integrator = sc.picardBDEIntegrator(absTol=10.0, convergenceVars=["ne", "ni", "Ge_dual", "Gi_dual", "We", "Wi"], internalStepControl=True)

rk.addIntegrator("BE", integrator)

initialTimestep=0.5
rk.setIntegratorGlobalData(initialTimestep=initialTimestep)

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE", bdeStep.dict())

We use `scalingTimestepController` to scale the time step so we always resolve the collisional time

In [ ]:
help(sc.scalingTimestepController)

The scaling time step controller scales the global time step duration by the minimum (or maximum) value of $c\prod_n v_n^{p_n}$(a product of variables raised to some powers) in the spatial domain. Remembering that collisional times scale as $T_e^{3/2}/n_e$, set the required variable names and powers in `scalingTimestepController` and use the `setTimestepController` function on the wrapper to add this time step control to the simulation.

In [33]:
rk.setTimestepController(sc.scalingTimestepController(reqVarNames=["ne", "Te"],reqVarPowers=[-1.0, 1.5]))

This will result in the time step being equal to the minimum value of $0.5T_e^{3/2}/n_e$ in normalized units ($t_0$ - the collisional time with $T_e=1$ and $n_e=1$)

Let's assume that, for whatever reason, we wish to run 3 periods of the perturbation ($300 \mu \text{s}$). We can do this by setting a real time target for the integration

In [34]:
rk.setTimeTargetTimestepping(3e-4,realTimeTarget=True)
rk.setMinimumIntervalOutput(100)

We also set the code to restart from the previous simulation, while resetting the value of the time variable to mimic starting from different initial conditions.

In [35]:
rk.setRestartOptions(save=False, # Save restart data
                     load=True, # Start from initial conditions and not from a saved restart file
                     resetTime=True) # Reset value of the time variable

It would have also been possible to directly restart from an hdf5 file using the `setHDF5FileInitialData` method of the wrapper. This is beyond the scope of this workshop, but it allows more flexibility since the number of MPI ranks can be changed, and variables can be added or removed from the simulation, which is particularly useful when adding diagnostics to already completed runs.

### Create config 

In [ ]:
rk.writeConfigFile()

### Data analysis

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

numFiles=42
loadpath = rk.hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedDataPert = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames,varsToIgnore=["ionGamma"])
loadedDataPert

Let's look at how the perturbation has affected the temperatures

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedDataPert,rk.grid)
dashboard.fluid2Comparison().show()

Use `fluidMultiComparison` to look at both the spatial profiles at each time as well as the temporal evolution at each spatial point of the electron and ion temperatures ("Te" and "Ti")

In [ ]:
# [YOUR CODE HERE]

In [ ]:
# [YOUR CODE HERE]